In [1]:
!pip install -q sahi ultralytics pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 31.9 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
%%writefile sahi_worker.py
import os
import json
import io
from contextlib import redirect_stdout
from tqdm import tqdm
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Salin path yang diperlukan ke sini agar worker bisa membacanya
# Atau pastikan variabel ini dikirim sebagai argumen (untuk simplifikasi, kita taruh sini)
DATASET_NAME = "widerface-baru"
BASE_DIR = f"/kaggle/input/{DATASET_NAME}/widerface"
VAL_IMAGES_DIR = os.path.join(BASE_DIR, "WIDER_val", "images")
VAL_JSON_PATH = os.path.join(BASE_DIR, "val_ground_truth_coco.json")
MODEL_NAME = "yolov11s-pose"
MODEL_PATH = f"/kaggle/input/{MODEL_NAME}/pytorch/default/1/yolo11s-pose-default/yolo11s_pose/weights/best.pt"
OUTPUT_DIR = "/kaggle/working"

def evaluate_sahi_on_gpu(gpu_id, param_combinations, output_queue):
    """
    Run evaluasi di GPU spesifik (Worker Function)
    """
    # Set GPU visibility
    os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
    
    print(f"\n🔥 GPU {gpu_id}: Loading model...")
    
    try:
        # Load model di GPU ini
        detection_model = AutoDetectionModel.from_pretrained(
            model_type='yolov8',
            model_path=MODEL_PATH,
            confidence_threshold=0.25,
            device="cuda:0"  # Selalu cuda:0 karena CUDA_VISIBLE_DEVICES sudah memfilter
        )
    except Exception as e:
        print(f"❌ GPU {gpu_id}: Error loading model: {e}")
        output_queue.put([])
        return
    
    # Load ground truth
    try:
        coco_gt = COCO(VAL_JSON_PATH)
    except Exception as e:
        print(f"❌ GPU {gpu_id}: Error loading GT: {e}")
        output_queue.put([])
        return
    
    img_ids = coco_gt.getImgIds()
    gpu_results = []
    
    # Process each parameter combination
    for idx, params in enumerate(param_combinations):
        print(f"\n🔧 GPU {gpu_id} [{idx+1}/{len(param_combinations)}]: "
              f"S{params['slice_size']} O{params['overlap_ratio']:.2f} "
              f"{params['postprocess_type']} {params['postprocess_match_metric']} "
              f"T{params['postprocess_match_threshold']:.2f}")
        
        results = []
        skipped = 0
        
        pbar = tqdm(img_ids, 
                   desc=f"GPU{gpu_id}",
                   position=gpu_id,
                   leave=False,
                   ncols=80)
        
        for img_id in pbar:
            img_info = coco_gt.loadImgs(img_id)[0]
            img_path = os.path.join(VAL_IMAGES_DIR, img_info['file_name'])
            
            if not os.path.exists(img_path):
                skipped += 1
                continue
            
            try:
                result = get_sliced_prediction(
                    img_path,
                    detection_model,
                    slice_height=params['slice_size'],
                    slice_width=params['slice_size'],
                    overlap_height_ratio=params['overlap_ratio'],
                    overlap_width_ratio=params['overlap_ratio'],
                    postprocess_type=params['postprocess_type'],
                    postprocess_match_metric=params['postprocess_match_metric'],
                    postprocess_match_threshold=params['postprocess_match_threshold'],
                    postprocess_class_agnostic=True,
                    verbose=0
                )
                
                for obj_pred in result.object_prediction_list:
                    bbox = obj_pred.bbox.to_xywh()
                    score = obj_pred.score.value
                    
                    results.append({
                        "image_id": img_id,
                        "category_id": 0,
                        "bbox": bbox,
                        "score": score
                    })
                
                pbar.set_postfix({'det': len(results), 'skip': skipped}, refresh=False)
                
            except Exception as e:
                skipped += 1
                continue
        
        pbar.close()
        
        if not results:
            print(f"❌ GPU {gpu_id}: No detections")
            gpu_results.append({
                "params": params,
                "metrics": {'map_50_95': 0.0, 'map_50': 0.0, 'map_75': 0.0}
            })
            continue
        
        # Evaluate
        try:
            # Gunakan unique filename agar tidak bentrok antar proses
            res_file = os.path.join(OUTPUT_DIR, 
                                   f"temp_gpu{gpu_id}_{params['slice_size']}_{params['overlap_ratio']}_{params['postprocess_type']}.json")
            with open(res_file, 'w') as f:
                json.dump(results, f)
            
            coco_dt = coco_gt.loadRes(res_file)
            coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
            
            with redirect_stdout(io.StringIO()):
                coco_eval.evaluate()
                coco_eval.accumulate()
                coco_eval.summarize()
            
            if os.path.exists(res_file):
                os.remove(res_file)
            
            metrics = {
                'map_50_95': coco_eval.stats[0],
                'map_50': coco_eval.stats[1],
                'map_75': coco_eval.stats[2]
            }
            
            print(f"✅ GPU {gpu_id}: mAP@50-95: {metrics['map_50_95']:.4f}")
            
            gpu_results.append({
                "params": params,
                "metrics": metrics
            })
            
        except Exception as e:
            print(f"❌ GPU {gpu_id}: Evaluation error: {e}")
            gpu_results.append({
                "params": params,
                "metrics": {'map_50_95': 0.0, 'map_50': 0.0, 'map_75': 0.0}
            })
    
    # Return results via queue
    output_queue.put(gpu_results)

Writing sahi_worker.py


In [ ]:
import os
import json
import itertools
import multiprocessing as mp
import sys
from datetime import datetime

# Tambahkan direktori saat ini ke path agar bisa import file yang baru dibuat
sys.path.append(os.getcwd())

# === IMPORT FUNGSI DARI FILE TERPISAH (SOLUSI FIX) ===
try:
    from sahi_worker import evaluate_sahi_on_gpu
except ImportError as e:
    print("❌ Error: Tidak bisa mengimport 'sahi_worker'.")
    print("Pastikan Anda sudah menjalankan cell '%%writefile sahi_worker.py' di atas!")
    raise e

# ================= KONFIGURASI PATH =================
# Path ini tetap didefinisikan di sini untuk keperluan VALIDASI di Main Process
# (Worker process akan menggunakan path yang ada di dalam sahi_worker.py)
DATASET_NAME = "widerface-baru"
BASE_DIR = f"/kaggle/input/{DATASET_NAME}/widerface"
VAL_IMAGES_DIR = os.path.join(BASE_DIR, "WIDER_val", "images")
VAL_JSON_PATH = os.path.join(BASE_DIR, "val_ground_truth_coco.json")
MODEL_NAME = "yolov11s-pose"
MODEL_PATH = f"/kaggle/input/{MODEL_NAME}/pytorch/default/1/yolo11s-pose-default/yolo11s_pose/weights/best.pt"
OUTPUT_DIR = "/kaggle/working"

# ================= GRID SEARCH =================
param_grid = {
    'slice_size': [320, 512, 640],
    'overlap_ratio': [0.3, 0.4],
    'postprocess_type': ['NMS'],
    'postprocess_match_metric': ['IOS', 'IOU'],
    'postprocess_match_threshold': [0.3, 0.5, 0.7]
}

# ================= MAIN PROGRAM =================
def main():
    print("="*70)
    print("KAGGLE MULTI-GPU SAHI TUNING (FIXED MULTIPROCESSING)")
    print("="*70)
    
    # ===== VALIDASI PATH =====
    print("\n📋 Validating paths (Main Process)...")
    
    print(f"\n1. Dataset Base Dir: {os.path.exists(BASE_DIR)}")
    print(f"2. Images Dir:       {os.path.exists(VAL_IMAGES_DIR)}")
    print(f"3. Ground Truth:     {os.path.exists(VAL_JSON_PATH)}")
    print(f"4. Model Path:       {os.path.exists(MODEL_PATH)}")
    
    if not all([os.path.exists(BASE_DIR), os.path.exists(VAL_IMAGES_DIR),
                os.path.exists(VAL_JSON_PATH), os.path.exists(MODEL_PATH)]):
        print("\n❌ CRITICAL ERROR: Path tidak ditemukan. Cek input dataset Kaggle Anda.")
        return
    
    print("\n✅ All paths validated!")
    print("="*70)
    
    # Cek jumlah GPU
    import torch
    n_gpus = torch.cuda.device_count()
    print(f"\n🔍 Detected GPUs: {n_gpus}")
    
    if n_gpus < 2:
        print("⚠️  Warning: Kurang dari 2 GPU terdeteksi. Script akan berjalan lambat (Single GPU).")
        n_gpus = 1
    
    # Generate combinations
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    print(f"📊 Total combinations: {len(combinations)}")
    
    if n_gpus > 1:
        # Split workload
        split_point = len(combinations) // 2
        combo_gpu0 = combinations[:split_point]
        combo_gpu1 = combinations[split_point:]
        
        print(f"🔀 GPU 0: {len(combo_gpu0)} combinations")
        print(f"🔀 GPU 1: {len(combo_gpu1)} combinations")
        print("="*70)
        
        # Create queue for results
        queue = mp.Queue()
        
        # Start processes
        # Argumen: (gpu_id, params, queue)
        p0 = mp.Process(target=evaluate_sahi_on_gpu, args=(0, combo_gpu0, queue))
        p1 = mp.Process(target=evaluate_sahi_on_gpu, args=(1, combo_gpu1, queue))
        
        start_time = datetime.now()
        
        print("\n🚀 Starting Worker Processes...")
        p0.start()
        p1.start()
        
        # Wait for completion & get results
        # Kita ambil result dulu sebelum join untuk menghindari deadlock jika queue penuh
        try:
            results_gpu0 = queue.get()
            results_gpu1 = queue.get()
        except Exception as e:
            print(f"❌ Error getting results from queue: {e}")
            p0.terminate()
            p1.terminate()
            return

        p0.join()
        p1.join()
        
        elapsed = (datetime.now() - start_time).total_seconds() / 60
        all_results = results_gpu0 + results_gpu1
        
    else:
        # Single GPU mode
        print("🔥 Running on single GPU (Direct Call)...")
        queue = mp.Queue()
        evaluate_sahi_on_gpu(0, combinations, queue)
        all_results = queue.get()
        elapsed = 0
    
    # Find best
    if not all_results:
        print("❌ No results collected.")
        return

    best_result = max(all_results, key=lambda x: x['metrics']['map_50_95'])
    
    # Report
    print("\n" + "="*70)
    print("HASIL AKHIR - TOP 10")
    print("="*70)
    
    sorted_results = sorted(all_results, key=lambda x: x['metrics']['map_50_95'], reverse=True)
    
    print(f"{'Rank':<5} {'Slice':<7} {'Over':<6} {'Post':<10} {'Metric':<6} {'Thresh':<7} {'mAP':<10}")
    print("-" * 70)
    
    for rank, r in enumerate(sorted_results[:10], 1):
        p = r['params']
        m = r['metrics']
        mark = "🏆" if rank == 1 else f"{rank}."
        print(f"{mark:<5} {p['slice_size']:<7} {p['overlap_ratio']:<6.2f} "
              f"{p['postprocess_type']:<10} {p['postprocess_match_metric']:<6} "
              f"{p['postprocess_match_threshold']:<7.2f} {m['map_50_95']:<10.4f}")
    
    print("-" * 70)
    print(f"\n🏆 BEST CONFIG:")
    print(f"   Slice Size               : {best_result['params']['slice_size']}")
    print(f"   Overlap Ratio            : {best_result['params']['overlap_ratio']}")
    print(f"   Post-process Type        : {best_result['params']['postprocess_type']}")
    print(f"   Post-process Metric      : {best_result['params']['postprocess_match_metric']}")
    print(f"   Post-process Threshold   : {best_result['params']['postprocess_match_threshold']}")
    print(f"   mAP@50-95                : {best_result['metrics']['map_50_95']:.4f}")
    
    if n_gpus > 1:
        print(f"\n⏱️  Total time: {elapsed:.1f} minutes ({elapsed/60:.1f} hours)")
    
    print("="*70)
    
    # Save results
    output_file = os.path.join(OUTPUT_DIR, "sahi_tuning_kaggle_results.json")
    with open(output_file, 'w') as f:
        json.dump({
            "best_params": best_result['params'],
            "best_metrics": best_result['metrics'],
            "all_results": all_results,
            "n_gpus_used": n_gpus,
            "total_time_minutes": elapsed if n_gpus > 1 else None
        }, f, indent=2)
    
    print(f"\n💾 Results: {output_file}")
    
    # Save best config
    config_file = os.path.join(OUTPUT_DIR, "best_sahi_config.json")
    with open(config_file, 'w') as f:
        json.dump(best_result['params'], f, indent=2)
    
    print(f"⚙️  Config  : {config_file}")

if __name__ == "__main__":
    # Set start method
    try:
        mp.set_start_method('spawn', force=True)
    except RuntimeError:
        pass
    main()

KAGGLE MULTI-GPU SAHI TUNING (FIXED MULTIPROCESSING)

📋 Validating paths (Main Process)...

1. Dataset Base Dir: True
2. Images Dir:       True
3. Ground Truth:     True
4. Model Path:       True

✅ All paths validated!

🔍 Detected GPUs: 2
📊 Total combinations: 120
🔀 GPU 0: 60 combinations
🔀 GPU 1: 60 combinations

🚀 Starting Worker Processes...

🔥 GPU 0: Loading model...
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!

🔧 GPU 0 [1/60]: S320 O0.10 NMS IOS T0.30

🔥 GPU 1: Loading model...
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!

🔧 GPU 1 [1/60]: S640 O0.10 NMS IOS T0.30


GPU0:  53%|███████▍      | 1724/3226 [20:06<10:38,  2.35it/s, det=29631, skip=0]

Loading and preparing results...
DONE (t=0.30s)
creating index...
index created!
✅ GPU 1: mAP@50-95: 0.3729

🔧 GPU 1 [2/60]: S640 O0.10 NMS IOS T0.50



GPU1:  94%|█████████████▏| 3044/3226 [18:18<01:02,  2.93it/s, det=36009, skip=0]
                                                                                
GPU1:  99%|█████████████▊| 3190/3226 [18:54<00:11,  3.02it/s, det=37994, skip=0]

Loading and preparing results...
DONE (t=0.33s)
creating index...
index created!
✅ GPU 0: mAP@50-95: 0.3372

🔧 GPU 0 [2/60]: S320 O0.10 NMS IOS T0.50


...


GPU1:  90%|████████████▌ | 2904/3226 [17:29<01:55,  2.78it/s, det=35406, skip=0]
                                                                                
GPU1:  95%|█████████████▎| 3058/3226 [18:06<00:43,  3.88it/s, det=36801, skip=0]

Loading and preparing results...
DONE (t=0.37s)
creating index...
index created!
✅ GPU 0: mAP@50-95: 0.3353

🔧 GPU 0 [3/60]: S320 O0.10 NMS IOS T0.70


GPU0:   5%|▊               | 171/3226 [01:38<21:25,  2.38it/s, det=6086, skip=0]

Loading and preparing results...
DONE (t=0.37s)
creating index...
index created!
✅ GPU 1: mAP@50-95: 0.3713

🔧 GPU 1 [5/60]: S640 O0.10 NMS IOU T0.50



GPU1:  97%|█████████████▌| 3121/3226 [18:38<00:40,  2.57it/s, det=39165, skip=0]